In [1]:
!pip install pympler -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.8/164.8 kB 3.3 MB/s eta 0:00:00


In [2]:
import seaborn as sns
import random
import numpy as np
import pandas as pd
import random
import numpy as np
from tqdm.auto import tqdm
import os
import time
import datetime


In [3]:
start = time.time()
save_path = '/content/drive/MyDrive/movie_rec/'
load_path = '/content/drive/MyDrive/movie_rec/'
df_user = pd.read_csv(load_path+"users.csv")
df_u = pd.get_dummies(df_user)
df_movie = pd.read_csv(load_path+"movies.csv")

# **Feature Engineering Movie**

In [4]:

def movie_feature(dff): #turn list of dictionary to list of strings with only the name
    import ast
    df = dff
    df = df.rename(columns={'id':'movie_id'})
    df['genres'] = dff['genres'].apply( lambda x: [i['name'] for i in ast.literal_eval(x)] )
    df['belongs_to_collection'] = dff['belongs_to_collection'].apply(lambda x:1 if x is not None else 0)
    df['production_countries'] = dff['production_countries'].apply(lambda x: 1 if 'United States of America' in x else 0)
    df['spoken_languages'] =     dff['spoken_languages'].apply(lambda x:1 if 'English' in x else 0)
    df['original_language'] = df['original_language'].astype(str)
    return df[df.columns.difference(['homepage','poster_path', 'tmdb_id', 'original_title','production_companies' ])]

def movie_data_fillna(df_movie_fea):
  df_movie_fea['status'] = df_movie_fea['status'].fillna('Released')
  pd.to_datetime(df_movie_fea['release_date']).mean()
  df_movie_fea['release_date'] = df_movie_fea['release_date'].fillna('2000-01-01')
  df_movie_fea['overview'] = df_movie_fea['overview'].fillna('None')
  df_movie_fea['imdb_id'] = df_movie_fea['imdb_id'].fillna('None')
  df_movie_fea[pd.isna(df_movie_fea['original_language'])]['original_language']='en'
  df_movie_fea['original_language'] = df_movie_fea['original_language'].apply(lambda x:x if type(x)==list else [x])
  df_movie_fea.isnull().sum()
  return df_movie_fea

def expand_list_to_rows(df, column_name): #expand columns with list of multiple values to multiple rows
    """
    Expand each list in the specified column of the dataframe into separate rows, while retaining the original index.
    """
    rows = []
    counter =0
    for i, row in df.iterrows():
        lists = row[column_name]
        if type(lists)==list:
          if lists != 'None' and len(lists)!=0:
              for item in lists:
                  new_row = row.to_dict()
                  new_row[column_name] = item
                  new_row['original_index'] = i  # Add original index to each row
                  rows.append(new_row)
          else:
              new_row = row.to_dict()
              new_row[column_name] = 'None'
              new_row['original_index'] = i
              rows.append(new_row)
    expanded_df = pd.DataFrame(rows)
    return expanded_df

def efficient_movie_feature_encoding(df, list_columns):
    """
    Efficiently one-hot encode columns that contain lists as values.
    """
    # First, handle normal features that don't need expansion
    useless_feature = ['imdb_id', 'overview', 'title', 'tmdb_id']
    df = df[df.columns.difference(useless_feature)]
    normal_features_df = df[df.columns.difference(list_columns + useless_feature)]

    # Then, for each list column, expand and one-hot encode it
    encoded_dfs = [normal_features_df]
    for column in tqdm(list_columns):
        expanded_df = expand_list_to_rows(df, column)
        encoded_df = pd.get_dummies(expanded_df, columns=[column], prefix=column)
        encoded_df = encoded_df[encoded_df.columns.difference(list_columns)]

        reduced_df = encoded_df.groupby('original_index').last()
        # Aggregate back to original DataFrame index
        encoded_dfs.append(reduced_df)

    final_df = pd.concat(encoded_dfs, axis=1)

    # Handle any duplicated columns due to concatenation, if necessary
    final_df = final_df.loc[:,~final_df.columns.duplicated()]

    return final_df

def movie_final_clean(df_m):
  df_m = df_m.drop(columns=['production_countries'])
  df_m['release_date'] = pd.to_datetime(df_m['release_date']).dt.year
  df_m['adult'] = df_m['adult'].astype(int)
  return df_m

In [5]:
def movie_feature_enigneering(df_movie):
  global df_movie_fea
  df_movie_fea = movie_feature(df_movie)
  df_movie_fea = movie_data_fillna(df_movie_fea)
  df_m =         efficient_movie_feature_encoding(df_movie_fea, ['genres','original_language'])
  df_m =         movie_final_clean(df_m)
  return df_m



In [6]:
df_m = movie_feature_enigneering(df_movie)
df_m.to_csv(save_path+"movie_all_features.csv")
df_m

  0%|          | 0/2 [00:00<?, ?it/s]

,adult,belongs_to_collection,budget,movie_id,popularity,release_date,revenue,runtime,spoken_languages,status,...,original_language_th,original_language_tl,original_language_tr,original_language_uk,original_language_ur,original_language_vi,original_language_wo,original_language_xx,original_language_zh,original_language_zu
0,0,1,0,nobody+else+but+you+2011,3.352027,2011,0,102,0,Released,...,0,0,0,0,0,0,0,0,0,0
1,0,1,50000000,alien+1992,17.126768,1992,159773545,114,1,Released,...,0,0,0,0,0,0,0,0,0,0
2,0,1,7000000,warlock+1989,11.906872,1989,0,103,1,Released,...,0,0,0,0,0,0,0,0,0,0
3,0,1,0,k-11+2013,1.686305,2013,0,88,1,Released,...,0,0,0,0,0,0,0,0,0,0
4,0,1,0,knuckle+2011,1.213981,2011,0,97,0,Released,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25673,0,1,0,touching+the+void+2003,7.594022,2003,13885802,106,1,Released,...,0,0,0,0,0,0,0,0,0,0
25674,0,1,0,winners++sinners+1983,2.272623,1983,0,108,0,Released,...,0,0,0,0,0,0,0,0,0,0
25675,0,1,0,salt+of+the+earth+1954,0.979701,1954,0,94,1,Released,...,0,0,0,0,0,0,0,0,0,0
25676,0,1,0,the+great+sacrifice+1944,0.128118,1944,0,98,0,Released,...,0,0,0,0,0,0,0,0,0,0


# **Feature Engineering User**

In [7]:
df_rating = pd.read_csv(load_path+"rate_log.csv").rename(columns={'minutes':'rating'})
df_watch = pd.read_csv(load_path+"watch_log.csv").rename(columns={'start_time':'time'})

In [8]:
df_full = df_rating.merge(df_u, 'left', on='user_id').fillna(0)
df_full = df_full.merge(df_movie_fea, 'left', left_on='movie_id', right_on='movie_id')
df_full = df_full.merge(df_watch[['user_id', 'movie_id', 'minutes']], 'left', on=['user_id', 'movie_id'], suffixes=('', '_watch')).fillna(0)


In [9]:
total_len = len(df_rating)
indices = list(range(total_len))
random.seed(42)
random.shuffle(indices)
split_point = int(0.75 * total_len)

training_indices = indices[:split_point]
test_indices = indices[split_point:]

data_train = df_full.iloc[training_indices]
data_test = df_full.iloc[test_indices]

In [10]:
user_watch_genre = df_full[['user_id', 'minutes','genres']]
user_watch_genre['genres'] = user_watch_genre['genres'].apply(lambda x:x[0] if (type(x)==list and len(x)>0) else 'None')
user_watch_genre =user_watch_genre.groupby(['user_id', 'genres'])['minutes'].mean().unstack(fill_value=0).reset_index()

user_rate_genre = df_full[['user_id', 'rating','genres']]
user_rate_genre['genres'] = user_rate_genre['genres'].apply(lambda x:x[0] if (type(x)==list and len(x)>0) else 'None')
user_rate_genre = user_rate_genre.groupby(['user_id', 'genres'])['rating'].count().unstack(fill_value=0).reset_index()


<ipython-input-10-f514d8a030c3>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_watch_genre['genres'] = user_watch_genre['genres'].apply(lambda x:x[0] if (type(x)==list and len(x)>0) else 'None')
<ipython-input-10-f514d8a030c3>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_rate_genre['genres'] = user_rate_genre['genres'].apply(lambda x:x[0] if (type(x)==list and len(x)>0) else 'None')


In [11]:
df_u_fea=df_u
df_u_fea = df_u_fea.merge(user_rate_genre, 'left', 'user_id', suffixes=('', '_rate')).fillna(0)
df_uu = df_u_fea.merge(user_watch_genre, 'left', 'user_id', suffixes=('_rate','_watch')).fillna(0)

In [12]:
df_uu.to_csv(save_path+'user_all_features.csv')
# df_mm = df_m[~df_m['id'].isin(['becoming+chaz+2011', 'the+butter+battle+book+1989', 'la+pointe-courte+1955', 'shoot+first_+die+later+1974'])]


In [13]:
data = df_rating.merge(df_uu, 'left', on='user_id').fillna(0)
data = data.merge(df_m, 'left', on='movie_id').fillna(0)

In [14]:
from sklearn.preprocessing import OrdinalEncoder
user_encoder = OrdinalEncoder()
user_encoder.fit(data['user_id'].values.reshape(-1, 1))
data['uid'] = user_encoder.transform(data['user_id'].values.reshape(-1, 1)).astype(int)

movie_encoder = OrdinalEncoder()
movie_encoder.fit(np.array(list(set(data['movie_id'].values).union(set(df_m['movie_id'].values)))).reshape(-1, 1))
data['mid'] = movie_encoder.transform(data['movie_id'].values.reshape(-1, 1)).astype(int)

In [15]:
import pickle
uid_user = [dict(enumerate(mapping)) for mapping in user_encoder.categories_][0]
mid_movie = [dict(enumerate(mapping)) for mapping in movie_encoder.categories_][0]
movie_mid = {v: k for k, v in mid_movie.items()}
user_uid = {v: k for k, v in uid_user.items()}
with open(load_path+'uid_user.pkl', 'wb') as f:
    pickle.dump(uid_user, f)
with open(load_path+'mid_movie.pkl', 'wb') as f:
    pickle.dump(mid_movie, f)
with open(load_path+'movie_mid.pkl', 'wb') as f:
    pickle.dump(movie_mid, f)
with open(load_path+'user_uid.pkl', 'wb') as f:
    pickle.dump(user_uid, f)
# movie_mapping

In [16]:
def date_encoding(data):
  data['day_of_week'] = pd.to_datetime(data['time']).dt.dayofweek.astype(int)
  data['is_weekend'] = (data['day_of_week'] >= 5).astype(int)
  data['hour'] = pd.to_datetime(data['time']).dt.hour.astype(int)
  data['hour_sin'] = np.sin(data['hour'] * (2. * np.pi / 24))
  data['hour_cos'] = np.cos(data['hour'] * (2. * np.pi / 24))
  data['day_of_week_sin'] = np.sin(data['day_of_week'] * (2. * np.pi / 7))
  data['day_of_week_cos'] = np.cos(data['day_of_week'] * (2. * np.pi / 7))
  data['release_date'] = pd.to_datetime(data['release_date']).dt.year
  return data.drop(columns=['time'])
data = date_encoding(data)
with pd.option_context('display.max_rows', 10, 'display.max_columns', None):
 display(data)

,user_id,movie_id,rating,age,occupation_K-12 student,occupation_academic/educator,occupation_artist,occupation_clerical/admin,occupation_college/grad student,occupation_customer service,occupation_doctor/health care,occupation_executive/managerial,occupation_farmer,occupation_homemaker,occupation_lawyer,occupation_other or not specified,occupation_programmer,occupation_retired,occupation_sales/marketing,occupation_scientist,occupation_self-employed,occupation_technician/engineer,occupation_tradesman/craftsman,occupation_unemployed,occupation_writer,gender_F,gender_M,Action_rate,Adventure_rate,Animation_rate,Comedy_rate,Crime_rate,Documentary_rate,Drama_rate,Family_rate,Fantasy_rate,Foreign_rate,History_rate,Horror_rate,Music_rate,Mystery_rate,None_rate,Romance_rate,Science Fiction_rate,TV Movie_rate,Thriller_rate,War_rate,Western_rate,Action_watch,Adventure_watch,Animation_watch,Comedy_watch,Crime_watch,Documentary_watch,Drama_watch,Family_watch,Fantasy_watch,Foreign_watch,History_watch,Horror_watch,Music_watch,Mystery_watch,None_watch,Romance_watch,Science Fiction_watch,TV Movie_watch,Thriller_watch,War_watch,Western_watch,adult,belongs_to_collection,budget,popularity,release_date,revenue,runtime,spoken_languages,status,vote_average,vote_count,genres_Action,genres_Adventure,genres_Animation,genres_Comedy,genres_Crime,genres_Documentary,genres_Drama,genres_Family,genres_Fantasy,genres_Foreign,genres_History,genres_Horror,genres_Music,genres_Mystery,genres_None,genres_Romance,genres_Science Fiction,genres_TV Movie,genres_Thriller,genres_War,genres_Western,original_language_ab,original_language_af,original_language_am,original_language_ar,original_language_ay,original_language_bg,original_language_bm,original_language_bn,original_language_bo,original_language_bs,original_language_ca,original_language_cn,original_language_cs,original_language_cy,original_language_da,original_language_de,original_language_el,original_language_en,original_language_eo,original_language_es,original_language_et,original_language_eu,original_language_fa,original_language_fi,original_language_fr,original_language_fy,original_language_gl,original_language_he,original_language_hi,original_language_hr,original_language_hu,original_language_id,original_language_is,original_language_it,original_language_ja,original_language_jv,original_language_ka,original_language_kk,original_language_kn,original_language_ko,original_language_ku,original_language_ky,original_language_la,original_language_lo,original_language_lt,original_language_lv,original_language_mk,original_language_ml,original_language_mn,original_language_mr,original_language_ms,original_language_nan,original_language_nb,original_language_ne,original_language_nl,original_language_no,original_language_pa,original_language_pl,original_language_ps,original_language_pt,original_language_qu,original_language_ro,original_language_ru,original_language_rw,original_language_sh,original_language_sk,original_language_sl,original_language_sq,original_language_sr,original_language_sv,original_language_ta,original_language_te,original_language_tg,original_language_th,original_language_tl,original_language_tr,original_language_uk,original_language_ur,original_language_vi,original_language_wo,original_language_xx,original_language_zh,original_language_zu,uid,mid,day_of_week,is_weekend,hour,hour_sin,hour_cos,day_of_week_sin,day_of_week_cos
0,160766,the+tigger+movie+2000,4,18,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,91.0,78.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,30000000.0,7.023414,1970,4.555453e+07,77.0,1.0,Released,6.3,146.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0

In [17]:
data['train'] = False
data.loc[training_indices,'train']=True
# data.to_csv(save_path+"train_test_data.csv")
import os
file_stats = os.stat(save_path+"train_test_data.csv")
print(file_stats.st_size / (1024 * 1024), "MB")

357.9247770309448 MB


In [18]:
def find_non_numerical_columns(df):
    non_numerical_columns = []
    for column in df.columns:
        # Attempt to convert column to numeric type. Unconvertible values are set to NaN
        temp = pd.to_numeric(df[column], errors='coerce')
        # If there are any NaN values in 'temp', it means the conversion was not fully possible
        if temp.isnull().any():
            non_numerical_columns.append(column)
    return non_numerical_columns
# find_non_numerical_columns(data)

In [19]:
# from sklearn.model_selection import train_test_split
# from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from xgboost import XGBRegressor

data_train = data[data['train']==True]
data_test = data[data['train']==False]

X_train = data_train[data_train.columns.difference(['movie_id','uid','user_id', 'status', 'rating', 'train'])]
y_train = data_train['rating']
X_test = data_test[data_test.columns.difference(['movie_id','uid','user_id', 'status', 'rating', 'train'])]
y_test = data_test['rating']


xgb_model = XGBRegressor()#max_depth=5, n_estimators=200)
xgb_model.fit(X_train, y_train)


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [20]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
train_pred =  xgb_model.predict(X_train)
mse = mean_squared_error(y_train,train_pred )
rmse = mse**0.5
mae = mean_absolute_error(y_train,train_pred )

print("Train metrics:")
print(f"MSE: {mse}")
print(f"RMSE: {rmse}")
print(f"MAE: {mae}")
print()

xgb_pred =  xgb_model.predict(X_test)
mse = mean_squared_error(y_test,xgb_pred)
rmse = mse**0.5
mae = mean_absolute_error(y_test,xgb_pred)

print("Test metrics:")
print(f"MSE: {mse}")
print(f"RMSE: {rmse}")
print(f"MAE: {mae}")

Train metrics:
MSE: 0.7696808030208864
RMSE: 0.8773145405274475
MAE: 0.6929054160527371

Test metrics:
MSE: 0.8317180591440841
RMSE: 0.9119857779286277
MAE: 0.7196214290643637


In [21]:
xgb_features = pd.DataFrame(columns=list(X_train.columns))
print("#XGBoost features:", len(list(X_train.columns)))
xgb_features.to_csv(save_path+"xgb_features.csv")

#XGBoost features: 188


In [22]:
import time
X_full = pd.concat([X_train, X_test])
y_full = pd.concat([y_train, y_test])
xgb_model = XGBRegressor()
start1 = time.time()
xgb_model.fit(X_full, y_full)
print('XGBoost Training time:', time.time()-start1, "Seconds")


xgb_model.save_model(save_path+'xgb_model.json')


from pympler import asizeof
import sys

file_stats = os.stat(save_path+"xgb_model.json")
print("XGBoost Model size on disk:", file_stats.st_size / (1024 * 1024), "MB")
approx_model_size = sys.getsizeof(xgb_model)
print("XGBoost approximate  Model size in RAM:", approx_model_size / (1024 * 1024), "MB")
model_size_bytes = asizeof.asizeof(xgb_model)
print(f"XGBoost more accurate Model size in RAM: {model_size_bytes/ (1024 * 1024)} MB")


XGBoost Training time: 24.3610200881958 Seconds
XGBoost Model size on disk: 0.644317626953125 MB
XGBoost approximate  Model size in RAM: 4.57763671875e-05 MB
XGBoost more accurate Model size in RAM: 0.00347137451171875 MB


In [23]:
print('Update time:', time.time()-start, "Seconds")

Update time: 96.61432123184204 Seconds


In [24]:
print("Training RAM cost:", round(5.4-0.8,2), "GB")

Training RAM cost: 4.6 GB
